<a href="https://colab.research.google.com/github/vn322/Netology_DS_course/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%D0%BF%D0%BE_%D1%82%D0%B5%D0%BC%D0%B5_%C2%AB%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BE%D0%B4%D0%B5%D1%80%D0%B6%D0%B0%D0%BD%D0%B8%D1%8F%C2%BB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание
1. Использовать dataset [MovieLens](https://grouplens.org/datasets/movielens/latest/)
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
    - TF-IDF на тегах и жанрах
    - Средние оценки (+ median, variance, etc.) пользователя и фильма
3. Оценить RMSE на тестовой выборке

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

from tqdm.notebook import tqdm

from sklearn.metrics import r2_score
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
links_df = pd.read_csv('links.csv')
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
tags_df = pd.read_csv('tags.csv')


In [ ]:
tags_df.head()


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
movies_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
tags_df['tag'] = tags_df['tag'] + ' '
tag_full = tags_df.groupby(['userId', 'movieId']).apply(lambda x: x.sum())
tag_full = tag_full['tag']
tag_full.head(3)

userId  movieId
2       60756           funny Highly quotable will ferrell 
        89774                   Boxing story MMA Tom Hardy 
        106782     drugs Leonardo DiCaprio Martin Scorsese 
Name: tag, dtype: object

In [ ]:
df = ratings_df.merge(movies_df, how='left', left_on=['movieId'], right_on=['movieId'])
df.head(3)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224.0,Heat (1995),Action|Crime|Thriller


In [ ]:
df = df.merge(tags_df, how='left', left_on=['userId', 'movieId'], right_on=['userId', 'movieId'])
df.head(3)

,userId,movieId,rating,timestamp_x,title,genres,tag,timestamp_y
0,1,1,4.0,964982703.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
1,1,3,4.0,964981247.0,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN
2,1,6,4.0,964982224.0,Heat (1995),Action|Crime|Thriller,NaN,NaN


In [ ]:
df = df.merge(tag_full, how='left', left_on=['userId', 'movieId'], right_index=True)
df.rename(columns = {'tag_x' : 'tag', 'tag_y' : 'tag_full',
                     'timestamp_x' : 'timestamp_rating', 'timestamp_y' : 'timestamp_tag'}, inplace = True)
df.head(3)
     

,userId,movieId,rating,timestamp_rating,title,genres,tag,timestamp_tag,tag_full
0,1,1,4.0,964982703.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,NaN
1,1,3,4.0,964981247.0,Grumpier Old Men (1995),Comedy|Romance,NaN,NaN,NaN
2,1,6,4.0,964982224.0,Heat (1995),Action|Crime|Thriller,NaN,NaN,NaN


In [ ]:
df['tag'].fillna('', inplace=True)
df['tag_full'].fillna('', inplace=True)
df['timestamp_tag'].fillna(0, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43501 entries, 0 to 43500
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   userId            43501 non-null  int64  
 1   movieId           43501 non-null  int64  
 2   rating            43501 non-null  float64
 3   timestamp_rating  43500 non-null  float64
 4   title             43501 non-null  object 
 5   genres            43501 non-null  object 
 6   tag               43501 non-null  object 
 7   timestamp_tag     43501 non-null  float64
 8   tag_full          43501 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 3.3+ MB


In [ ]:
df['genres'] = df['genres'].apply(lambda x: x.replace('|', ' '))
df.loc[:, 'genres_tag_full'] = df.loc[:, 'genres'] + ' ' + df.loc[:, 'tag_full']
df.loc[:, 'genres_tag'] = df.loc[:, 'genres'] + ' ' + df.loc[:, 'tag']
df.head(3)

,userId,movieId,rating,timestamp_rating,title,genres,tag,timestamp_tag,tag_full,genres_tag_full,genres_tag
0,1,1,4.0,964982703.0,Toy Story (1995),Adventure Animation Children Comedy Fantasy,,0.0,,Adventure Animation Children Comedy Fantasy,Adventure Animation Children Comedy Fantasy
1,1,3,4.0,964981247.0,Grumpier Old Men (1995),Comedy Romance,,0.0,,Comedy Romance,Comedy Romance
2,1,6,4.0,964982224.0,Heat (1995),Action Crime Thriller,,0.0,,Action Crime Thriller,Action Crime Thriller


In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(['rating'], axis=1)
y = df['rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=X['userId'])
print(X_train.shape)
print(X_test.shape)
     

(39150, 10)
(4351, 10)


In [ ]:
users = pd.concat([X_train,y_train], axis=1).groupby('userId').mean('rating')
users = users['rating'].to_dict()

movies = pd.concat([X_train,y_train], axis=1).groupby('movieId').mean('rating')
movies = movies['rating'].to_dict()

In [ ]:
X_train['users'] = X_train['userId'].apply(lambda x: users[x])
X_train['movies'] = X_train['movieId'].apply(lambda x: movies[x])
X_train.head(3)

,userId,movieId,timestamp_rating,title,genres,tag,timestamp_tag,tag_full,genres_tag_full,genres_tag,users,movies
3328,21,2858,1.441393e+09,American Beauty (1999),Drama Romance,,0.0,,Drama Romance,Drama Romance,3.274436,4.080000
30050,202,2355,9.749239e+08,"Bug's Life, A (1998)",Adventure Animation Children Comedy,,0.0,,Adventure Animation Children Comedy,Adventure Animation Children Comedy,3.818182,3.652778
36770,248,116823,1.534592e+09,The Hunger Games: Mockingjay - Part 1 (2014),Adventure Sci-Fi Thriller,,0.0,,Adventure Sci-Fi Thriller,Adventure Sci-Fi Thriller,3.739130,3.954545


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

tfidf = TfidfVectorizer()
X_train = pd.concat([pd.DataFrame(tfidf.fit_transform(X_train['genres_tag_full']).toarray(), 
                                  columns=tfidf.get_feature_names_out()).reset_index(drop=True), 
                     X_train['users'].reset_index(drop=True),
                     X_train['movies'].reset_index(drop=True)
                     ],
                    axis=1)
ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)
y_train
print(X_train.shape)

X_test['users'] = X_test['userId'].apply(lambda x: users[x])
X_test['movies'] = X_test['movieId'].apply(lambda x: movies.get(x, 3.5))  # Add average for all movies
X_test = pd.concat([pd.DataFrame(tfidf.transform(X_test['genres_tag']).toarray(), 
                                columns=tfidf.get_feature_names_out()).reset_index(drop=True),
                    X_test['users'].reset_index(drop=True),
                    X_test['movies'].reset_index(drop=True)
                    ],
                   axis=1)
ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)
y_test
print(X_test.shape)

(39150, 536)
(4351, 536)


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, y_train)


KNeighborsRegressor(n_neighbors=3)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, knn.predict(X_test), squared=False)**0.5

0.9948676045896445

In [ ]:
from sklearn import metrics

metrics.r2_score(y_test, knn.predict(X_test))

0.06515525792608123